In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import conrad
import cvxpy
import numpy as np
import matplotlib

from matplotlib import pyplot as plt
from conrad import Case

In [ ]:
# Define dimensions of problem
m_target = 100
m_oar = 400
m = m_target + m_oar
n = 200

# Structure labels
lab_tum = 0
lab_oar = 1

# Generate random beam matrix
A_target = np.random.rand(m_target, n)
A_oar = 0.5 * np.random.rand(m_oar, n)
A = np.vstack((A_target, A_oar))

label_order = [lab_tum, lab_oar]
voxel_labels = [lab_tum] * m_target + [lab_oar] * m_oar

In [ ]:
# Prescription for each structure
rx = [{'label': lab_tum, 'name': 'tumor', 'is_target': True,  'dose': 1., 'constraints': None},
      {'label': lab_oar, 'name': 'oar',   'is_target': False, 'dose': 0., 'constraints': None}]

In [ ]:
# Construct and solve case with no DVH constraints
cs = Case(A, voxel_labels, label_order, rx)
cs.plan("ECOS", "dvh_no_slack", verbose = 1)

In [ ]:
# Add DVH constraints and resolve case
cs.add_dvh_constraint(lab_tum, 1.05, 0.3, '<')
cs.add_dvh_constraint(lab_tum, 0.8, 0.2, '>')
cs.add_dvh_constraint(lab_oar, 0.5, 0.5, '<')

In [ ]:
# Solve without slacks using a single pass
cs.plan("ECOS", "dvh_no_slack", plot = True)

In [ ]:
# Solve without slacks using 2 passes
cs.plan("ECOS", "dvh_no_slack", "dvh_2pass", plot = True)

In [ ]:
# Additional DVH constraint makes no-slack problem infeasible
cs.add_dvh_constraint(lab_oar, 0.55, 0.1, '>')

In [ ]:
# Solving without slacks will result in infeasibility
cs.plan("ECOS", "dvh_no_slack")

In [ ]:
# Solve with slacks using 2 passes
cs.plan("ECOS", "dvh_2pass", plot = True)